In [1]:
from kaggle_environments import evaluate, make, utils
import os
import sys
import inspect
import time
import numpy as np
import torch
import gymnasium as gym
from tianshou.algorithm.modelfree.dqn import DiscreteQLearningPolicy

from Submissions.mcts_agent import mcts_agent
from Submissions.minimax_agent import minimax_agent
from Submissions.rainbow_agent import rainbow_agent

env = make("connectx", debug=True)
print(env.render(mode="ansi"))

[kaggle_environments.envs.open_spiel_env.open_spiel_env] INFO: Successfully loaded OpenSpiel environments: 8.
[kaggle_environments.envs.open_spiel_env.open_spiel_env] INFO:    open_spiel_chess
[kaggle_environments.envs.open_spiel_env.open_spiel_env] INFO:    open_spiel_connect_four
[kaggle_environments.envs.open_spiel_env.open_spiel_env] INFO:    open_spiel_gin_rummy
[kaggle_environments.envs.open_spiel_env.open_spiel_env] INFO:    open_spiel_go
[kaggle_environments.envs.open_spiel_env.open_spiel_env] INFO:    open_spiel_tic_tac_toe
[kaggle_environments.envs.open_spiel_env.open_spiel_env] INFO:    open_spiel_universal_poker
[kaggle_environments.envs.open_spiel_env.open_spiel_env] INFO:    open_spiel_repeated_poker
[kaggle_environments.envs.open_spiel_env.open_spiel_env] INFO:    open_spiel_python_repeated_pokerkit
[kaggle_environments.envs.open_spiel_env.open_spiel_env] INFO: OpenSpiel games skipped: 0.
+---+---+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+--

In [2]:
# Random agent
def random_agent(observation, configuration):
    from random import choice
    return choice([c for c in range(configuration.columns) if observation.board[c] == 0])

In [3]:
# Evaluating agent against random and negamax
def mean_reward(rewards, role):
    if role == "P1":
        return np.mean([r[0] for r in rewards])
    else:
        return np.mean([r[1] for r in rewards])

def evaluate_agent(agent, num_episodes=10):
    rewards_random_p1 = evaluate("connectx", [agent, "random"], num_episodes=num_episodes)
    rewards_random_p2 = evaluate("connectx", ["random", agent], num_episodes=num_episodes)
    rewards_negamax_p1 = evaluate("connectx", [agent, "negamax"], num_episodes=num_episodes)
    rewards_negamax_p2 = evaluate("connectx", ["negamax", agent], num_episodes=num_episodes)

    print(f"Evaluating agent against random and negamax with {num_episodes} games as P1 and P2...\n")
    
    print(f"P1 vs Random: {mean_reward(rewards_random_p1, 'P1')}")
    print(f"P1 vs Negamax: {mean_reward(rewards_negamax_p1, 'P1')}\n")  

    print(f"P2 vs Random: {mean_reward(rewards_random_p2, 'P2')}")
    print(f"P2 vs Negamax: {mean_reward(rewards_negamax_p2, 'P2')}")

In [4]:
# Debugging agent against random
def debug_agent(agent):
    env = make("connectx", debug=True)
    trainer = env.train([None, "random"])
    observation = trainer.reset()

    while not env.done:
        t0 = time.time()
        my_action = agent(observation, env.configuration)
        t1 = time.time()
        print(f"Turn {observation.step + 1}: Action {my_action} (in {t1 - t0:.2f} seconds)")
        observation, reward, done, info = trainer.step(my_action)   

    print(f"\nYou won :)\n" if env.state[0].reward > 0 else "\nYou lost :(\n")
    print(env.render(mode="ansi"))

In [17]:
# Facing agents
def facing_agents(agent_a, agent_b, num_episodes=10):
    a_rewards = []
    b_rewards = []

    env = make("connectx", debug=True)

    for i in range(num_episodes):
        if i < num_episodes // 2:
            env.run([agent_a, agent_b])
            a_rewards.append(env.state[0].reward)
            b_rewards.append(env.state[1].reward)
        else:
            env.run([agent_b, agent_a])
            b_rewards.append(env.state[0].reward)
            a_rewards.append(env.state[1].reward)

    print(f"Agent A won {a_rewards.count(1)} times")
    print(f"Agent B won {b_rewards.count(1)} times")
    print(f"Draw {a_rewards.count(0)} times")
    return a_rewards, b_rewards

In [6]:
# Writing agent to file and validating submission format
def write_agent_to_file(agent):
    file = agent.__name__ + ".py"
    with open(file, "a" if os.path.exists(file) else "w") as f:
        f.write(inspect.getsource(agent))

def validate_submission(agent):
    out = sys.stdout
    agent = utils.read_file(agent.__name__ + ".py")
    sys.stdout = out

    env = make("connectx", debug=True)
    env.run([agent, agent])

    print("Success :)" if env.state[0].status == env.state[1].status == "DONE" else "Failed :(")

In [25]:
# Facing different agents
print("Facing MCTS vs Minimax")
res = facing_agents(mcts_agent, minimax_agent)
print(res)

print("\nFacing MCTS vs Rainbow")
res = facing_agents(mcts_agent, rainbow_agent)
print(res)

print("\nFacing Minimax vs Rainbow")
res = facing_agents(minimax_agent, rainbow_agent)
print(res)

Facing MCTS vs Minimax
Agent A won 9 times
Agent B won 0 times
Draw 1 times
([1, 1, 1, 1, 1, 1, 0, 1, 1, 1], [-1, -1, -1, -1, -1, -1, 0, -1, -1, -1])

Facing MCTS vs Rainbow
Agent A won 10 times
Agent B won 0 times
Draw 0 times
([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1])

Facing Minimax vs Rainbow
Agent A won 10 times
Agent B won 0 times
Draw 0 times
([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1])


In [ ]:
# Debugging and evaluating Minimax
debug_agent(minimax_agent)
evaluate_agent(minimax_agent)

NameError: name 'debug_agent' is not defined

In [ ]:
# Debugging and evaluating MCTS
debug_agent(mcts_agent)
evaluate_agent(mcts_agent)

Turn 1: Action 3 (in 1.90 seconds)
Turn 3: Action 3 (in 1.90 seconds)
Turn 5: Action 5 (in 1.90 seconds)
Turn 7: Action 3 (in 1.90 seconds)
Turn 9: Action 3 (in 1.90 seconds)

You won :)

+---+---+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+---+---+
| 0 | 0 | 0 | 1 | 0 | 0 | 0 |
+---+---+---+---+---+---+---+
| 0 | 0 | 0 | 1 | 0 | 0 | 0 |
+---+---+---+---+---+---+---+
| 0 | 0 | 0 | 1 | 0 | 1 | 0 |
+---+---+---+---+---+---+---+
| 2 | 2 | 2 | 1 | 0 | 2 | 0 |
+---+---+---+---+---+---+---+



In [ ]:
# Debugging and evaluating Rainbow
debug_agent(rainbow_agent)
evaluate_agent(rainbow_agent)



Evaluating agent against random and negamax with {num_episodes} games as P1 and P2...
P1 vs Random: 1.0
P1 vs Negamax: 1.0
P2 vs Random: 1.0
P2 vs Negamax: 1.0


In [12]:
# Facing different agents
print("Facing MCTS vs Minimax")
facing_agents(mcts_agent, minimax_agent)

print("\nFacing Minimax vs MCTS")
facing_agents(minimax_agent, mcts_agent)



print("\nFacing Minimax vs Rainbow")
facing_agents(minimax_agent, rainbow_agent)

print("\nFacing MCTS vs Rainbow")
facing_agents(mcts_agent, rainbow_agent)

Facing MCTS vs Minimax
Agent A won 10 times
Agent B won 0 times
Draw 0 times

Facing Minimax vs MCTS
Invalid Action: Invalid column: 0
Agent A won 0 times
Agent B won 9 times
Draw 0 times

Facing Minimax vs Rainbow
Invalid Action: Invalid column: 0
Invalid Action: Invalid column: 0
Invalid Action: Invalid column: 0
Invalid Action: Invalid column: 0
Invalid Action: Invalid column: 0
Invalid Action: Invalid column: 0
Invalid Action: Invalid column: 0
Invalid Action: Invalid column: 0
Invalid Action: Invalid column: 0
Invalid Action: Invalid column: 0
Agent A won 0 times
Agent B won 0 times
Draw 0 times

Facing MCTS vs Rainbow
Agent A won 10 times
Agent B won 0 times
Draw 0 times


In [ ]:
# Creating agents submissions and validating them
# write_agent_to_file(minimax_agent)
validate_submission(minimax_agent)

# write_agent_to_file(mcts_agent)
validate_submission(mcts_agent)

# write_agent_to_file(rainbow_agent)
validate_submission(rainbow_agent)

Success :)
Success :)
